In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pymongo import MongoClient
from pandas.io.json import json_normalize
import re
from pprint import pprint
import string

# ============================================ renmin fake

In [2]:
r = pd.read_csv('renmin.csv')
r.drop(['Unnamed: 0', '_id_y'], axis=1, inplace=True)
r.drop_duplicates(inplace=True)
r.drop(['crawl_time_x', 'crawl_time_y', 'created_at', 'org_wbid_uid', 'repost_user_id'], axis=1, inplace=True)
r.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6420 entries, 0 to 7857
Data columns (total 17 columns):
_id_x                 6420 non-null int64
authentication        1088 non-null object
birthday              5027 non-null object
brief_introduction    5676 non-null object
city                  2543 non-null object
fans_num              6420 non-null int64
follows_num           6420 non-null int64
gender                6412 non-null object
labels                2522 non-null object
nick_name             6412 non-null object
province              6412 non-null object
sentiment             208 non-null object
sex_orientation       145 non-null object
tweets_num            6420 non-null int64
vip_level             6412 non-null object
content               6420 non-null object
repost_device         4962 non-null object
dtypes: int64(4), object(13)
memory usage: 902.8+ KB


In [3]:
r.head()

,_id_x,authentication,birthday,brief_introduction,city,fans_num,follows_num,gender,labels,nick_name,province,sentiment,sex_orientation,tweets_num,vip_level,content,repost_device
0,2849560641,NaN,巨蟹座,人生在世，谁都会吃屎，但是别嚼,成都,84,163,女,美食,芒果奶熙,四川,NaN,NaN,114,未开通,芒果奶熙:尊重是一切最基本的遵循//@罗云熙Leo:尊重是一切最基本的遵循 赞[0] 03月...,来自 iPhone客户端
1,2849560641,NaN,巨蟹座,人生在世，谁都会吃屎，但是别嚼,成都,84,163,女,美食,芒果奶熙,四川,NaN,NaN,114,未开通,芒果奶熙:尊重是一切最基本的遵循//@罗云熙Leo:尊重是一切最基本的遵循 赞[0] 03月...,来自 iPhone客户端
2,2315828514,NaN,1900-12-29,NaN,NaN,32,61,女,旅游,心随纸鸢自在飞,上海,NaN,NaN,1565,3级,心随纸鸢自在飞:有你@罗云熙Leo 相伴，不惧风雨。快来为TA应援吧 //@罗云熙Leo:...,来自明星ALL榜
3,2315828514,NaN,1900-12-29,NaN,NaN,32,61,女,旅游,心随纸鸢自在飞,上海,NaN,NaN,1565,3级,心随纸鸢自在飞:有你@罗云熙Leo 相伴，不惧风雨。快来为TA应援吧 //@罗云熙Leo:...,来自明星ALL榜
4,6459746842,NaN,NaN,NaN,NaN,3,104,男,NaN,用户6459746842,其他,NaN,NaN,2,未开通,用户6459746842://@六小龄童:自取其辱 赞[0] 2018-12-20 15:5...,来自荣耀Note10 大屏旗舰


In [4]:
r_yonghu = list(r.loc[((r['nick_name'].str.contains('^用户', na=False)) & (r['vip_level'] == '未开通')), 'nick_name'].unique())
r_yonghu

['用户6459746842',
 '用户6841613130',
 '用户6178461401',
 '用户6673393309',
 '用户6291509410',
 '用户6494032131',
 '用户5616685914',
 '用户3948702009',
 '用户5667874413',
 '用户6623674717']

In [5]:
r_fake = r[((r['follows_num']<=5)|(r['fans_num']<=5))&
                 (r['birthday'].isna())&
                 (r['city'].isna())&
                (r['sentiment'].isna())&
                (r['sex_orientation'].isna())&
                (r['brief_introduction'].isna())].copy()
r_fake.drop_duplicates(inplace=True)
r_fake.shape

(10, 17)

In [6]:
r_fake = list(r_fake['nick_name'].unique())
r_fake

['用户6459746842',
 '我觉得你不行丶',
 '用户6291509410',
 '用户6494032131',
 'Tony的战衣w',
 '用户5616685914',
 '晓芸在江湖',
 '丹巴吉美',
 '蜻蜓FM音乐课代表']

In [7]:
r_digit = r[r['nick_name'].str.contains(r'\d+$', na=False)].copy().drop_duplicates()
r_digit.shape

(370, 17)

In [8]:
r_digit = list(r_digit['nick_name'].unique())
r_digit

['用户6459746842',
 '莎莎的梦11',
 '用户6841613130',
 '女汉子完美585',
 '傅-1127',
 '腾飞的骏马48761',
 '叶伶886',
 '用户6178461401',
 '心兴星7',
 '小棚子1017',
 '哈哈就是我699',
 '要开心鸭1688',
 'Seoman1999',
 'July720',
 'enough_97735',
 '安静待97',
 '艺神66666',
 '么么哒闯天涯009',
 '山花精1208',
 '下凡的天使0313',
 '磨磨叽叽77',
 'Demian0215',
 '小草的爱6452491085',
 '有点纠结的_527',
 '爱幂不解释_24777',
 '羊羊羊200909',
 'Doraemon753',
 'Dear-佛系美少女88',
 '万小鹿1997',
 '遇见pp叫a1',
 '偷吃芒果111',
 'DUYIYAN11',
 '且行且珍惜74594',
 '咕噜咕噜兔7',
 'Love如果0828',
 'yhm-1987',
 'ww20160911',
 '随便看看就好24',
 '超级源-LBY685',
 '不知名的小学生575',
 '我鱼贼啦6',
 'Purple_0713',
 '旺仔的小馒头0131',
 '谁的青春乱了流年_97621',
 'Jessica无所谓_1988',
 '红莲1007',
 'Peonypeony521',
 '与尤长靖一起有长进的2019',
 '弯呢菜菜56286',
 '夏天的天4711',
 '许豆豆9',
 'yumi0710',
 '密码不能忘55',
 '别来关注886',
 '鱼说1979',
 '婧婧婧婧婧婧2',
 '李妮12',
 '盗号好玩吗20151223',
 '粥喵吃鲸鱼123',
 '柠檬酸甜酱123',
 '杰2c1',
 'isghkshdb14',
 '李昇炫的大米1',
 '晓辉0106',
 '叶莉9699',
 '最好的你112233',
 '柠檬柚子921',
 '小九NiNe99',
 '册册_7',
 '不要真情实感411',
 '橙子味仙女520',
 '现充积分87',
 'Rachelzhang0227',
 '大白44444

In [9]:
r['nick_name'].isna().sum()

8

In [10]:
r['nick_name'].fillna(value='noname', inplace=True)
r_eng_char = list(r.loc[r['nick_name'].map(lambda v: any(char in v for char in string.punctuation)), 'nick_name'].unique())
r_eng_char      

['blingblingblinglingling_',
 '傅-1127',
 '周淋浴-',
 '莫说---浅浅淡淡F',
 'M24_头号粉丝',
 '宁-墨-',
 'Me_酷girl',
 '上善若水_YS',
 '离别真如我的_',
 'Icey--潇潇',
 '缓缓-早睡早起',
 'hi_emmahoff',
 '他是一棵树_',
 'Syd-Paradise',
 'DreamyCrush_',
 'FOUR-sixteen',
 '飞鸟和鱼_L',
 'Luminiita_KK',
 '牛奶炖星星_',
 '郑怪怪-',
 '绒花_',
 '__Yuki_雪',
 '姝烟倾菀豆_',
 'enough_97735',
 '范二青年-瑞同学',
 'lion_M沫璐M_deer',
 'Cassiopeia看着神起_期待合体',
 'Dear_小兔',
 'journey_to',
 '北卡蓝_Jan',
 'luckyandlucky_',
 '本宫已经睡醒了_',
 '是骄傲啊-',
 '来一杯冰阔乐__',
 '啵丝猫_',
 'zzt-小蚌壳fy',
 '超爱尤长靖的LBJ仙女_',
 '麦田里的风声-',
 '不加番茄酱_',
 '有点纠结的_527',
 '白滚滚_FOR',
 'Dear-高伟光',
 'Tracy_Nano',
 '爱幂不解释_24777',
 'Helius-K-',
 '人潮湧動我只愛你_',
 '_草莓樱桃啦啦啦_',
 '牛奶烧仙草_',
 'kun_靖',
 '清风自来_心若',
 '鍾爱_维尼',
 '木子洋的小耳朵-ONERGAL',
 'Dear-佛系美少女88',
 '桃之夭夭__灼灼紫华',
 '_内内内内内个谁',
 '離_歸_',
 'ningoo_',
 '乂十七_',
 '诗若灯青-',
 '芒果_猫猫',
 '四点五十九-',
 '-平---',
 '诺贝尔超可爱奖_',
 '_嗨虾',
 '时落晞-YU',
 '-怀中猫g',
 '骆骆骆骆驼_祥子',
 '斑比-MoMo-默默',
 'xxxx雅-',
 'Simona_Ww',
 'Killer兴_Xback',
 '会BOOM的唐_',
 'Noemie_xue',
 '十年伴添福_双儿',
 '慧啊慧-吖',
 '-缱绻宇宙',

In [11]:
charlist = list(set('（亻彳讠饣艹丶乛亠冖宀冫丷扌氵灬丨亅丿乚勹匚冂凵爫礻钅廴辶犭卩刂阝疒纟丬忄飠釒糹歺虍耂牜癶夂覀罒衤亅乛乁乀）（卐彡乄冫）（亻彳讠饣艹丶乛亠冖宀冫丷扌氵灬丨亅丿乚勹匚冂凵爫礻钅廴辶犭卩刂阝疒纟丬忄飠釒糹歺虍耂牜癶夂覀罒衤亅乛乁乀）（卐彡乄冫）'))

r_chi_char = []
for c in charlist:
    tmp = r[r['nick_name'].str.contains(c, na=False)].copy().drop_duplicates()
    names = list(tmp['nick_name'].unique())
    r_chi_char += names
r_chi_char = list(set(r_chi_char))
r_chi_char

['丫丶旧-',
 '豆丶豆豆豆豆子',
 '小芋圆罒3罒',
 '茕茕白兔丨朱一龙',
 '超丨甜',
 '王足各口阝可',
 '惊鸿照影来彡',
 '一片绯色的鱼丶',
 '女王丶病娇',
 '鑫xin灬灬',
 'Angie丶甜',
 '大丈夫萌呆乃丶丶丶',
 '小小小兔酱丶',
 'Regina丨梁只赢',
 '毕丨之',
 '八十一哟丶',
 'a丶sHero',
 '我觉得你不行丶',
 '晓不语丶',
 '怼怼呐丨毕竟又长大一岁啦',
 '威崽丶',
 '我儿今天解约了吗丶',
 'Oni丨',
 '夜神樱良丷',
 '二艹_你是我的奇迹',
 'Imirror丶d']

In [12]:
r_bd = list(r.loc[r['birthday'].str.contains('^201', na=False), 'nick_name'].unique())
len(r_bd)

198

In [13]:
r_fanhui = list(r.loc[((r['content'].str.contains('^返回')) | (r['content'].str.contains('//'))) 
                                           & (r['brief_introduction'].isna()), 'nick_name'].unique())
len(r_fanhui)

148

In [14]:
r_names = list(set(r_yonghu + r_fake + r_digit + r_chi_char + r_bd + r_fanhui))
len(r_names)

672

# ============================================ zhang fake

In [15]:
z = pd.read_csv('zhang_merged.csv')
z.drop(['Unnamed: 0', '_id_y'], axis=1, inplace=True)
z.drop_duplicates(inplace=True)
z.drop(['crawl_time_x', 'crawl_time_y', 'created_at', 'org_wbid_uid', 'repost_user_id'], axis=1, inplace=True)

In [16]:
z.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7308 entries, 0 to 14042
Data columns (total 17 columns):
_id_x                 7308 non-null int64
authentication        610 non-null object
birthday              3882 non-null object
brief_introduction    2758 non-null object
city                  2468 non-null object
fans_num              7307 non-null float64
follows_num           7307 non-null float64
gender                7308 non-null object
labels                791 non-null object
nick_name             7308 non-null object
province              7308 non-null object
sentiment             326 non-null object
sex_orientation       60 non-null object
tweets_num            7307 non-null float64
vip_level             7308 non-null object
content               7308 non-null object
repost_device         580 non-null object
dtypes: float64(3), int64(1), object(13)
memory usage: 1.0+ MB


In [17]:
# ============================================ nick name
z_yonghu = z[z['nick_name'].str.contains('^用户', na=False)].copy().drop_duplicates()

In [18]:
z_yonghu['_id_x'].value_counts().shape

(91,)

In [19]:
z_yonghu = list(z_yonghu['nick_name'].unique())
z_yonghu

['用户6751917874',
 '用户6750984191',
 '用户6751965975',
 '用户6751021484',
 '用户6736814689',
 '用户6900763324',
 '用户6765418070',
 '用户6827930453',
 '用户6910108534',
 '用户6828412357',
 '用户6660880069',
 '用户6883315543',
 '用户6743016204',
 '用户6825553246',
 '用户6739943318',
 '用户6730618127',
 '用户6828508506',
 '用户6825485780',
 '用户6919882792',
 '用户6909893146',
 '用户6909989062',
 '用户6731550940',
 '用户1275819459',
 '用户6644481368',
 '用户6640489191',
 '用户6910813024',
 '用户6609949238',
 '用户6910108927',
 '用户6597476627',
 '用户6637906121',
 '用户6942303688',
 '用户6861909724',
 '用户6606014755',
 '用户4332331901',
 '用户6595131151',
 '用户6578846885',
 '用户6601311079',
 '用户6596767475',
 '用户6618780085',
 '用户6619279824',
 '用户6618778723',
 '用户6825431710',
 '用户6618781133',
 '用户6609778071',
 '用户6741913667',
 '用户6919876349',
 '用户6599692700',
 '用户6603022863',
 '用户6825374791',
 '用户6912548707',
 '用户6654010984',
 '用户6654615616',
 '用户6765387890',
 '用户6828649157',
 '用户6827429814',
 '用户6742814114',
 '用户6918514901',
 '用户6715246929',
 '用户6741992925

In [20]:
z_fake = z[((z['follows_num']<=5)|(z['fans_num']<=5))&
                 (z['birthday'].isna())&
                 (z['city'].isna())&
                (z['sentiment'].isna())&
                (z['sex_orientation'].isna())&
                (z['brief_introduction'].isna())].copy()
z_fake.drop_duplicates(inplace=True)
z_fake.shape

(145, 17)

In [21]:
z_fake = list(z_fake['nick_name'].unique())
z_fake

['用户6900763324',
 '绝美耀文25576',
 '天宇可爱36611',
 '暖男坤哥73006',
 '用户6910108534',
 '用户6660880069',
 '刈其楚言tT3627',
 '用户6919882792',
 '用户6909893146',
 '用户6909989062',
 '用户1275819459',
 '用户6644481368',
 '用户6640489191',
 '用户6910813024',
 '用户6609949238',
 '用户6910108927',
 '用户6597476627',
 '用户6637906121',
 '用户6942303688',
 '用户6606014755',
 '用户4332331901',
 '用户6595131151',
 '用户6578846885',
 '用户6609778071',
 '用户6919876349',
 '热血难凉筱1995',
 '用户6603022863',
 '用户6912548707',
 '用户6654010984',
 '用户6654615616',
 '用户6918514901',
 '用户6647237400',
 '用户6912477437',
 '用户6500326079',
 '用户6609351835',
 '用户6911190133',
 '用户6606500557',
 '用户6597509127',
 '用户6606628795',
 '向上向善Ny',
 '用户6661339720',
 '用户6920779139',
 '至真至纯霜玢1997灬',
 '用户6646054075',
 '用户6920789707',
 '用户6920779168',
 '用户6911822444',
 '单恋detective020',
 '用户6606600578',
 '用户6646040998',
 'N3丶盐',
 '用户6911834590',
 '用户6646546106',
 '用户6577782283',
 '至纯至真丶HW']

In [22]:
z_digit = z[z['nick_name'].str.contains(r'\d+$', na=False)].copy().drop_duplicates()
z_digit.shape

(367, 17)

In [23]:
z_digit = list(z_digit['nick_name'].unique())
z_digit

['用户6751917874',
 '用户6750984191',
 '用户6751965975',
 '用户6751021484',
 '心兴星7',
 'SHAN92808',
 '用户6736814689',
 '无去处8',
 '秋雨梧桐86_507',
 '用户6900763324',
 '用户6765418070',
 '用户6827930453',
 '绝美耀文25576',
 '天宇可爱36611',
 '暖男坤哥73006',
 '用户6910108534',
 '霸王龙LX1007',
 '爱到这72070',
 '用户6828412357',
 '坚持98832',
 '麻婆豆腐35085',
 '催眠术08417',
 '小葫芦sohappy12968',
 '快门回溯30895',
 '用户6660880069',
 '相55555',
 '用户6883315543',
 'constraint200007',
 '用户6743016204',
 '用户6825553246',
 '用户6739943318',
 '用户6730618127',
 '用户6828508506',
 '刈其楚言tT3627',
 '用户6825485780',
 '用户6919882792',
 '用户6909893146',
 '望兴1007',
 '用户6909989062',
 '望兴19911007',
 '用户6731550940',
 '独角兔子10',
 '用户1275819459',
 '如果当初的我勇敢qj86',
 '用户6644481368',
 '用户6640489191',
 '用户6910813024',
 '用户6609949238',
 '用户6910108927',
 '人蕾动1007',
 'Tattoo107',
 '集结号168',
 '集结号107',
 '用户6597476627',
 '用户6637906121',
 '用户6942303688',
 'magnitude201003',
 'tar097',
 'noted200406',
 '用户6861909724',
 '用户6606014755',
 '蜡笔x坨107',
 '用户4332331901',
 '用户6595131151',
 '用户6578

In [24]:
charlist = list(set('（亻彳讠饣艹丶乛亠冖宀冫丷扌氵灬丨亅丿乚勹匚冂凵爫礻钅廴辶犭卩刂阝疒纟丬忄飠釒糹歺虍耂牜癶夂覀罒衤亅乛乁乀）（卐彡乄冫）（亻彳讠饣艹丶乛亠冖宀冫丷扌氵灬丨亅丿乚勹匚冂凵爫礻钅廴辶犭卩刂阝疒纟丬忄飠釒糹歺虍耂牜癶夂覀罒衤亅乛乁乀）（卐彡乄冫）'))
pprint(charlist)

['乛',
 '卐',
 '冫',
 '彡',
 '廴',
 '虍',
 '乄',
 '亻',
 '覀',
 '丬',
 '冂',
 '讠',
 '亠',
 '釒',
 '爫',
 '（',
 '凵',
 '灬',
 '乁',
 '艹',
 '疒',
 '匚',
 '牜',
 '丿',
 '钅',
 '衤',
 '罒',
 '忄',
 '饣',
 '耂',
 '卩',
 '礻',
 '）',
 '阝',
 '刂',
 '夂',
 '飠',
 '宀',
 '糹',
 '癶',
 '扌',
 '乀',
 '乚',
 '亅',
 '歺',
 '冖',
 '勹',
 '丶',
 '彳',
 '丷',
 '丨',
 '纟',
 '犭',
 '辶',
 '氵']


In [25]:
z_chi_char = []
for c in charlist:
    tmp = z[z['nick_name'].str.contains(c, na=False)].copy().drop_duplicates()
    names = list(tmp['nick_name'].unique())
    z_chi_char += names
z_chi_char = list(set(z_chi_char))
z_chi_char

['前运颇股劫灬',
 '至纯至真丶HW',
 'N3丶盐',
 '十号少年FR丶',
 '至纯至真灬Lattimore',
 '大大大大大大大迷糊丶',
 'LQ温润如玉丶',
 '至真至纯霜玢1997灬',
 '霞-罒-罒-']

In [26]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [27]:
z_eng_char = list(z.loc[z['nick_name'].map(lambda v: any(char in v for char in string.punctuation)), 'nick_name'].unique())
z_eng_char      
#       ['nick_name', 'fans_num', 'follows_num', 'tweets_num', 'vip_level', 'brief_introduction', 'repost_device']].drop_duplicates()

['xback-艺兴一意只为你',
 'X-back在水一方',
 'AutumnM_xback',
 '和b_TuttiMay',
 '宓e_lhwant',
 '木_南街',
 'Xback_cindy苏苏',
 'Monologue-Lay',
 'xback-sasha',
 '霞-罒-罒-',
 '秋雨梧桐86_507',
 '恩恩恩兴L_XBack',
 '和平的平--春晓的晓',
 'healing_unicorn_',
 'Xback-TX',
 'Ling1007-紫海城洛灵兮',
 'PRETTYIXING-',
 '冷小姐_L',
 '啾_咪Y',
 'YIiiw_',
 'Xback_YUSA的欺诈银狐',
 '巢m_gyhq',
 '辣椒炒肉Bd_',
 '蕏瘢哋洊茬_kmwc',
 '-SHARPAYyyyy',
 'ss199212138-',
 '南风__life',
 'Xback_ivy',
 'Monkey欧巴_',
 '嫣然_AmiJk',
 '雅痞_wxtwr',
 '雨落流殇_amgclx',
 '符f_MonicaMi',
 'ISSING-XBACK',
 '简简单单_xback',
 '岁月无声也让人害怕-xss',
 '毛of_UVICFiona',
 '砷陕_dtffzm',
 'Xback-颖',
 '南巷_zuopiezi',
 '余q_michiko',
 '从s_ViviNeil',
 '真的太甜-',
 '国际人渣_SexKou',
 '拥暖如初_dedienr',
 '巴黎左岸_getwayp',
 '隐藏所有情绪_',
 '别怼我_我怂',
 'YSJ-GM',
 '属于我们的秘密_甜兴']

In [28]:
z_names = list(set(z_yonghu + z_fake + z_digit + z_chi_char + z_eng_char))
len(z_names)

186

In [29]:
# ============================================ other
z.loc[z['nick_name'].isin(z_yonghu), 'birthday'].value_counts()

1969-03-07    160
1979-06-13    140
2014-02-20    102
1989-09-19     96
2005-12-15     73
1960-01-01     48
1994-10-23     44
2012-02-13     36
2007-04-10     35
2004-01-21     34
2005-01-05     30
2004-02-05     27
1966-02-05     25
1992-10-22     24
2003-10-07     23
2013-01-28     21
2010-04-08     12
1965-02-04     10
1997-09-25      9
2013-02-13      7
1995-07-19      7
1994-10-22      6
1999-12-04      5
1967-03-05      5
2001-02-22      4
2009-11-05      4
2008-03-13      4
2006-04-14      2
2016-06-23      1
1996-04-05      1
1983-07-16      1
1999-03-08      1
Name: birthday, dtype: int64

In [30]:
z[z['birthday'].str.contains('^201', na=False)]

,_id_x,authentication,birthday,brief_introduction,city,fans_num,follows_num,gender,labels,nick_name,province,sentiment,sex_orientation,tweets_num,vip_level,content,repost_device
12,6751021484,NaN,2016-06-23,NaN,贺州,1.0,0.0,女,NaN,用户6751021484,广西,NaN,NaN,49.0,未开通,用户6751021484: 赞[0] 03月03日 22:49 来自Android,来自Android
186,5253754956,NaN,2016-07-01,Bi 尊重 勿忘初心…,NaN,84.0,156.0,女,NaN,心兴星7,其他,NaN,NaN,21781.0,未开通,心兴星7://@UnicornLay_张艺兴打榜基地:#制作人张艺兴# #张艺兴出席格莱美颁...,来自 iPhone客户端
187,5253754956,NaN,2016-07-01,Bi 尊重 勿忘初心…,NaN,84.0,156.0,女,NaN,心兴星7,其他,NaN,NaN,21781.0,未开通,返回心兴星7的微博,NaN
189,5253754956,NaN,2016-07-01,Bi 尊重 勿忘初心…,NaN,84.0,156.0,女,NaN,心兴星7,其他,NaN,NaN,21781.0,未开通,返回心兴星7的微博,NaN
191,5253754956,NaN,2016-07-01,Bi 尊重 勿忘初心…,NaN,84.0,156.0,女,NaN,心兴星7,其他,NaN,NaN,21781.0,未开通,返回心兴星7的微博,NaN
193,5253754956,NaN,2016-07-01,Bi 尊重 勿忘初心…,NaN,84.0,156.0,女,NaN,心兴星7,其他,NaN,NaN,21781.0,未开通,返回心兴星7的微博,NaN
195,5253754956,NaN,2016-07-01,Bi 尊重 勿忘初心…,NaN,84.0,156.0,女,NaN,心兴星7,其他,NaN,NaN,21781.0,未开通,返回心兴星7的微博,NaN
197,5253754956,NaN,2016-07-01,Bi 尊重 勿忘初心…,NaN,84.0,156.0,女,NaN,心兴星7,其他,NaN,NaN,21781.0,未开通,返回心兴星7的微博,NaN
199,5253754956,NaN,2016-07-01,Bi 尊重 勿忘初心…,NaN,84.0,156.0,女,NaN,心兴星7,其他,NaN,NaN,21781.0,未开通,返回心兴星7的微博,NaN
201,5253754956,NaN,2016-07-01,Bi 尊重 勿忘初心…,NaN,84.0,156.0,女,NaN,心兴星7,其他,NaN,NaN,21781.0,未开通,返回心兴星7的微博,NaN


In [31]:
z_bd = list(z.loc[z['birthday'].str.contains('^201', na=False), 'nick_name'].unique())
len(z_bd)

13

In [32]:
z[((z['content'].str.contains('^返回')) | (z['content'].str.contains('//'))) 
                                           & (z['brief_introduction'].isna())]

,_id_x,authentication,birthday,brief_introduction,city,fans_num,follows_num,gender,labels,nick_name,province,sentiment,sex_orientation,tweets_num,vip_level,content,repost_device
8,6751917874,NaN,1999-12-04,NaN,新加坡,1.0,0.0,女,NaN,用户6751917874,海外,NaN,NaN,54.0,未开通,返回用户6751917874的微博,NaN
36,5699804869,NaN,NaN,NaN,其他,177.0,167.0,女,NaN,X-back在水一方,台湾,NaN,NaN,58187.0,未开通,X-back在水一方://@UnicornLay_张艺兴打榜基地:#制作人张艺兴# #张艺兴...,来自 Android客户端
37,5699804869,NaN,NaN,NaN,其他,177.0,167.0,女,NaN,X-back在水一方,台湾,NaN,NaN,58187.0,未开通,返回X-back在水一方的微博,NaN
95,5856130264,NaN,处女座,NaN,NaN,130.0,198.0,女,NaN,紫韵成风,其他,NaN,NaN,2924.0,未开通,紫韵成风://@UnicornLay_张艺兴打榜基地:#制作人张艺兴# #张艺兴出席格莱美颁...,来自海信H11 AI拍你更美
141,6222795155,NaN,NaN,NaN,NaN,110.0,165.0,男,NaN,绿地都,四川,NaN,NaN,13700.0,未开通,绿地都://@UnicornLay_张艺兴打榜基地:#制作人张艺兴# #张艺兴出席格莱美颁奖...,来自三星A8s黑瞳全视屏
142,6222795155,NaN,NaN,NaN,NaN,110.0,165.0,男,NaN,绿地都,四川,NaN,NaN,13700.0,未开通,返回绿地都的微博,NaN
144,6222795155,NaN,NaN,NaN,NaN,110.0,165.0,男,NaN,绿地都,四川,NaN,NaN,13700.0,未开通,返回绿地都的微博,NaN
146,6222795155,NaN,NaN,NaN,NaN,110.0,165.0,男,NaN,绿地都,四川,NaN,NaN,13700.0,未开通,返回绿地都的微博,NaN
148,6222795155,NaN,NaN,NaN,NaN,110.0,165.0,男,NaN,绿地都,四川,NaN,NaN,13700.0,未开通,返回绿地都的微博,NaN
150,6222795155,NaN,NaN,NaN,NaN,110.0,165.0,男,NaN,绿地都,四川,NaN,NaN,13700.0,未开通,返回绿地都的微博,NaN


In [33]:
z_fanhui = list(z.loc[((z['content'].str.contains('^返回')) | (z['content'].str.contains('//'))) 
                                           & (z['brief_introduction'].isna()), 'nick_name'].unique())
len(z_fanhui)

170

In [34]:
z_names = list(set(z_names + z_bd + z_fanhui))
len(z_names)

216

# ============================================ 懒得搞def

In [122]:
# zfdf = z[z['nick_name'].isin(z_names)].copy()
zfdf = zauth.copy()
zfdf.shape

(1500, 17)

In [123]:
cat_names = ['default_name_cat', 'garbled_name_cat', 'en_name_cat', 'ch_name_cat']
name_lists = [z_yonghu, z_digit, z_eng_char, z_chi_char]

for i in range(4):
    zfdf[cat_names[i]] = 0
    zfdf.loc[zfdf['nick_name'].isin(name_lists[i]), cat_names[i]] = 1
    print('\n=====' + cat_names[i])
    print(zfdf[cat_names[i]].value_counts())


=====default_name_cat
0    1500
Name: default_name_cat, dtype: int64

=====garbled_name_cat
0    1500
Name: garbled_name_cat, dtype: int64

=====en_name_cat
0    1500
Name: en_name_cat, dtype: int64

=====ch_name_cat
0    1500
Name: ch_name_cat, dtype: int64


In [124]:
zfdf['vip_level'].isna().sum()

0

In [125]:
zfdf['vip_level'].fillna(0, inplace=True)

In [126]:
# zfdf[zfdf.repost_device.str.contains('榜', na=False)]
devices = zfdf['repost_device'].value_counts().rename_axis('unique_values').reset_index(name='counts')
common_devices = list(devices.loc[devices['counts'] > 1, 'unique_values'])

In [127]:
zfdf['bi_device'] = 0
zfdf.loc[~zfdf['repost_device'].isin(common_devices), 'bi_device'] = 1
zfdf['bi_device'].value_counts()

1    1411
0      89
Name: bi_device, dtype: int64

In [128]:
zfdf.loc[zfdf['tweets_num'] < 1500, ['nick_name', 'vip_level', 'fans_num', 'follows_num', 'tweets_num']]\
    .drop_duplicates()\
    .sort_values(by='tweets_num')

,nick_name,vip_level,fans_num,follows_num,tweets_num
336,爱张艺兴的贝壳女孩,1级,260.0,180.0,300.0
3705,方脸困的对床,未开通,36.0,195.0,451.0
1333,小宝贝张艺兴,2级,62.0,178.0,514.0
2012,牵着小蜗去看海,未开通,59.0,168.0,563.0
1067,黄小九啵,未开通,163.0,167.0,978.0
3295,艺颗夜空中最亮的兴,3级,107.0,239.0,1043.0
1149,Xback的宝贝兴,未开通,8.0,19.0,1167.0
7237,一丢丢甜x,1级,105.0,76.0,1326.0
293,晶年夏天,未开通,82.0,142.0,1411.0


In [129]:
# comment out if sampling auth!!!!
# zfdf = zfdf[zfdf['tweets_num'] < 1500].copy()
# print(len(zfdf['nick_name'].unique()))
# print(zfdf.shape)

In [130]:
zfdf.province.value_counts()

湖南     284
北京     264
其他     209
海外     184
广东     112
浙江      61
安徽      50
天津      45
新疆      44
福建      43
辽宁      41
黑龙江     39
上海      23
江西      18
河北      18
陕西      15
重庆      14
广西       9
河南       7
吉林       7
四川       4
山西       4
山东       3
香港       2
Name: province, dtype: int64

In [131]:
# zfdf.head()

In [132]:
cols = ['authentication', 'brief_introduction', 'city', 'sentiment', 'sex_orientation']
for col in cols:
    zfdf['bi_' + col] = 0
    zfdf.loc[~zfdf[col].isna(), 'bi_' + col] = 1
    print('\n====== bi_' + col)
    print(zfdf['bi_' + col].value_counts())


====== bi_authentication
0    1161
1     339
Name: bi_authentication, dtype: int64

====== bi_brief_introduction
1    1500
Name: bi_brief_introduction, dtype: int64

====== bi_city
0    997
1    503
Name: bi_city, dtype: int64

====== bi_sentiment
0    1322
1     178
Name: bi_sentiment, dtype: int64

====== bi_sex_orientation
0    1453
1      47
Name: bi_sex_orientation, dtype: int64


In [133]:
zfdf['bi_bd'] = 1
zfdf.loc[zfdf['birthday'].str.contains('^201', na=False), 'bi_bd'] = 0
zfdf.loc[zfdf['birthday'].isna(), 'bi_bd'] = -1
zfdf['bi_bd'].value_counts()

 1    1071
-1     429
Name: bi_bd, dtype: int64

In [134]:
zfdf['bi_labels'] = 0
zfdf.loc[~zfdf['birthday'].isna(), 'bi_labels'] = 1
zfdf['bi_labels'].value_counts()

1    1071
0     429
Name: bi_labels, dtype: int64

In [135]:
zfdf.drop(zfdf[zfdf.vip_level==0].index, inplace=True)

In [136]:
zfdf.vip_level.value_counts()

未开通    539
3级     256
6级     202
1级     172
5级     161
2级     140
4级      30
Name: vip_level, dtype: int64

In [137]:
zfdf['vip_level'] = zfdf['vip_level'].map({'未开通': 0,
                       '1级': 1,
                       '2级': 2,
                       '3级': 3,
                       '4级': 4,
                       '5级': 5,
                       '6级': 6
#                                            , '7级': 7
                                          })
zfdf['vip_level'].value_counts()

0    539
3    256
6    202
1    172
5    161
2    140
4     30
Name: vip_level, dtype: int64

In [138]:
zfdf['bi_vip'] = 0
zfdf.loc[zfdf['vip_level'] > 0, 'bi_vip'] = 1
zfdf['bi_vip'].value_counts()

1    961
0    539
Name: bi_vip, dtype: int64

In [139]:
zfdf['bi_content'] = 0
zfdf.loc[zfdf['content'].str.contains('^返回', na=False), 'bi_content'] = 1
zfdf.loc[zfdf['content'].str.contains('//', na=False), 'bi_content'] = 1
zfdf['bi_content'].value_counts()

1    1499
0       1
Name: bi_content, dtype: int64

In [140]:
zfdf['province'].value_counts()

湖南     284
北京     264
其他     209
海外     184
广东     112
浙江      61
安徽      50
天津      45
新疆      44
福建      43
辽宁      41
黑龙江     39
上海      23
江西      18
河北      18
陕西      15
重庆      14
广西       9
河南       7
吉林       7
四川       4
山西       4
山东       3
香港       2
Name: province, dtype: int64

In [141]:
zfdf['bi_province'] = 0
zfdf.loc[((zfdf['province'] == '海外') | (zfdf['province'] == '其他')), 'bi_province'] = 1
zfdf.bi_province.value_counts()

0    1107
1     393
Name: bi_province, dtype: int64

In [142]:
zfdf.gender.unique()

array(['女', '男'], dtype=object)

In [143]:
len(zfdf[zfdf.nick_name == 'noname'].index)

0

In [144]:
zfdf.drop(zfdf[zfdf.nick_name == 'noname'].index, inplace=True)

In [145]:
zfdf['gender'] = zfdf['gender'].map({'女': 1, '男': 0})
zfdf.gender.value_counts()

1    1424
0      76
Name: gender, dtype: int64

In [146]:
zfdf['followers_cat'] = 0
zfdf.loc[zfdf['fans_num'] <= 1, 'followers_cat'] = 1
zfdf['followers_cat'].value_counts()
# ============================================ fake account (w/ FP) characters

0    1500
Name: followers_cat, dtype: int64

In [62]:
pprint(zfdf.columns.tolist())

['_id_x',
 'authentication',
 'birthday',
 'brief_introduction',
 'city',
 'fans_num',
 'follows_num',
 'gender',
 'labels',
 'nick_name',
 'province',
 'sentiment',
 'sex_orientation',
 'tweets_num',
 'vip_level',
 'content',
 'repost_device',
 'default_name_cat',
 'garbled_name_cat',
 'en_name_cat',
 'ch_name_cat',
 'bi_device',
 'bi_authentication',
 'bi_brief_introduction',
 'bi_city',
 'bi_sentiment',
 'bi_sex_orientation',
 'bi_bd',
 'bi_labels',
 'bi_vip',
 'bi_content',
 'bi_province',
 'followers_cat']


In [147]:
# out = zfdf[['_id_x',
#  'fans_num',
#  'follows_num',
#  'gender',
#  'tweets_num',
#  'vip_level',
#  'default_name_cat',
#  'garbled_name_cat',
#  'en_name_cat',
#  'ch_name_cat',
#  'bi_device',
#  'bi_authentication',
#  'bi_brief_introduction',
#  'bi_vip',
#  'bi_content',
#  'followers_cat']].copy()

out = zfdf[['_id_x',
 'fans_num',
 'follows_num',
 'gender',
 'tweets_num',
 'vip_level',
 'default_name_cat',
 'garbled_name_cat',
 'en_name_cat',
 'ch_name_cat',
 'bi_device',
 'bi_authentication',
 'bi_brief_introduction',
 'bi_city',
 'bi_sentiment',
 'bi_sex_orientation',
 'bi_bd',
 'bi_labels',
 'bi_vip',
 'bi_content',
 'bi_province',
 'followers_cat']].copy()

# # order
# out = out[['_id_x', 
#            'tweets_num', 
#            'fans_num', 
#            'follows_num', 
#            'vip_level', 
#            'gender', 
#            'default_name_cat', 
#            'garbled_name_cat',
#            'en_name_cat', 
#            'ch_name_cat', 
#            'followers_cat', 
#            'bi_device', 
#            'bi_authentication', 
#            'bi_vip', 
#            'bi_content', 
#            'bi_brief_introduction']]
# # change col names
# out.columns = ['id', 
#                'posts_num', 
#                'followers_num', 
#                'followings_num', 
#                'vip_num', 
#                'gender', 
#                'default_name_cat', 
#                'garbled_name_cat', 
#                'en_name_cat', 
#                'ch_name_cat', 
#                'followers_cat', 
#                'device_cat', 
#                'verified_cat', 
#                'vip_cat', 
#                'default_cont_cat', 
#                'intro_cat']

# # out.drop_duplicates(inplace=True)
out.shape

(1500, 22)

In [148]:
out.to_csv('z_auth_plus.csv', index=False)

In [ ]:
id, 

posts_num 发微博数
followers_num 粉丝数
followings_num 关注数 
vip_num 会员等级

gender: 0 male 1 female

default_name_cat, 
garbled_name_cat, 
en_name_cat, 
ch_name_cat, 

followers_cat: 粉丝数1或者0 → 1
device_cat: 独立设计的 → 1
verified_cat: 认证过 → 1
vip_cat: 会员 → 1
default_cont_cat: unique['raw_text']==('转发微博', '轉發微博','转发') → 1
intro_cat: 有介绍→ 1

# ============================================ sample authentic

In [93]:
print(r[~r['nick_name'].isin(z_names)].shape, z[~z['nick_name'].isin(z_names)].shape)

(6418, 17) (1545, 17)


In [94]:
rauth = r[~r['nick_name'].isin(r_names)].sample(4500)
zauth = z[~z['nick_name'].isin(z_names)].sample(1500)

# ============================================ merge 2 scrapers

In [150]:
rfake = pd.read_csv('./traintestdata/r_fake_plus.csv')
zfake = pd.read_csv('./traintestdata/z_fake_plus.csv')
rauth = pd.read_csv('./traintestdata/r_auth_plus.csv')
zauth = pd.read_csv('./traintestdata/z_auth_plus.csv')

In [388]:
clfake = pd.read_csv('./traintestdata/fake_cai_luo.csv')
clauth = pd.read_csv('./traintestdata/true_cai_luo.csv')

In [390]:
clfake.drop('Unnamed: 0', axis=1, inplace=True)
clauth.drop('Unnamed: 0', axis=1, inplace=True)
print(clfake.shape, clauth.shape)

(998, 16) (7383, 16)


In [151]:
fake = pd.concat([rfake, zfake], ignore_index=True).sample(frac=1)
auth = pd.concat([rauth, zauth], ignore_index=True).sample(frac=1)
print(fake.shape, auth.shape)

(4411, 22) (6000, 22)


In [152]:
fake['Effective'] = 0
auth['Effective'] = 1
out = pd.concat([fake, auth], ignore_index=True).sample(frac=1)
out.shape

(10411, 23)

In [153]:
out.head()

,_id_x,fans_num,follows_num,gender,tweets_num,vip_level,default_name_cat,garbled_name_cat,en_name_cat,ch_name_cat,...,bi_city,bi_sentiment,bi_sex_orientation,bi_bd,bi_labels,bi_vip,bi_content,bi_province,followers_cat,Effective
3942,6730618127,1.0,5.0,0,61.0,0,1,1,0,0,...,1,0,0,1,1,0,1,0,1,0
5067,1979866143,618.0,205.0,1,8077.0,1,0,0,0,0,...,1,0,0,1,1,1,0,0,0,1
5705,6047176678,32.0,136.0,1,236.0,0,0,0,0,0,...,0,0,0,1,1,0,1,0,0,1
3093,1947719030,20.0,43.0,1,782.0,1,0,0,1,0,...,0,0,0,1,1,1,1,1,0,0
6444,6495821902,269.0,215.0,1,664.0,3,0,0,0,0,...,0,0,0,1,1,1,1,0,0,1


In [154]:
out.to_csv('weibo_data_simple.csv', index=False)